# Quickstart

This is a self contained guide on how to write a simple app and start launching
distributed jobs on local and remote clusters.

## Installation

First thing we need to do is to install the TorchX python package which includes
the CLI and the library.

```sh
# install torchx with all dependencies
$ pip install torchx[dev]
```

See the [README](https://github.com/pytorch/torchx) for more
information on installation.

In [1]:
%%sh
torchx --help

usage: torchx [-h] [--log_level LOG_LEVEL] [--version]


              {builtins,cancel,configure,describe,list,log,run,runopts,status,tracker}


              ...


torchx CLI


optional arguments:


...

                        Python logging log level


  --version             show program's version number and exit


sub-commands:


  Use the following commands to run operations, e.g.: torchx run ${JOB_NAME}


  {builtins,cancel,configure,describe,list,log,run,runopts,status,tracker}


## Hello World

Lets start off with writing a simple "Hello World" python app. This is just a
normal python program and can contain anything you'd like.

<div class="admonition note">
<div class="admonition-title">Note</div>
This example uses Jupyter Notebook `%%writefile` to create local files for
example purposes. Under normal usage you would have these as standalone files.
</div>

In [2]:
%%writefile my_app.py

import sys

print(f"Hello, {sys.argv[1]}!")

Overwriting my_app.py


## Launching

We can execute our app via `torchx run`. The
`local_cwd` scheduler executes the app relative to the current directory.

For this we'll use the `utils.python` component:

In [3]:
%%sh
torchx run --scheduler local_cwd utils.python --help

usage: torchx run <run args...> python  [--help] [-m M] [-c C]


                                        [--script SCRIPT] [--image IMAGE]


                                        [--name NAME] [--cpu CPU] [--gpu GPU]


                                        [--memMB MEMMB] [-h H]


                                        [--num_replicas NUM_REPLICAS]


...

  -h H, --h H           a registered named resource (if specified takes


                        precedence over cpu, gpu, memMB) (default: None)


  --num_replicas NUM_REPLICAS


                        number of copies to run (each on its own container)


                        (default: 1)


The component takes in the script name and any extra arguments will be passed to
the script itself.

In [4]:
%%sh
torchx run --scheduler local_cwd utils.python --script my_app.py "your name"

torchx 2023-02-19 07:58:57 INFO     Log directory not set in scheduler cfg. Creating a temporary log dir that will be deleted on exit. To preserve log directory set the `log_dir` cfg option


torchx 2023-02-19 07:58:57 INFO     Log directory is: /tmp/torchx_5i6mjkkf


torchx 2023-02-19 07:58:57 INFO     Waiting for the app to finish...


python/0 Hello, your name!


torchx 2023-02-19 07:58:58 INFO     Job finished: SUCCEEDED


local_cwd://torchx/torchx_utils_python-f9mlsfdz2v1dx


We can run the exact same app via the `local_docker` scheduler. This scheduler
will package up the local workspace as a layer on top of the specified image.
This provides a very similar environment to the container based remote
schedulers.

<div class="admonition note">
<div class="admonition-title">Note</div>
This requires Docker installed and won't work in environments such as Google
Colab. See the Docker install instructions:
[https://docs.docker.com/get-docker/](https://docs.docker.com/get-docker/)</a>
</div>

In [5]:
%%sh
torchx run --scheduler local_docker utils.python --script my_app.py "your name"

torchx 2023-02-19 07:58:59 INFO     Checking for changes in workspace `file:///home/runner/work/torchx/torchx/docs/source`...


torchx 2023-02-19 07:58:59 INFO     To disable workspaces pass: --workspace="" from CLI or workspace=None programmatically.


torchx 2023-02-19 07:58:59 INFO     Workspace `file:///home/runner/work/torchx/torchx/docs/source` resolved to filesystem path `/home/runner/work/torchx/torchx/docs/source`


torchx 2023-02-19 07:58:59 INFO     Building workspace docker image (this may take a while)...


torchx 2023-02-19 07:59:09 INFO     Built new image `sha256:e99a43099771ef952d2513ead430ac233a25f142a7f5abc0ad44e0bd463695a3` based on original image `ghcr.io/pytorch/torchx:0.5.0dev0` and changes in workspace `file:///home/runner/work/torchx/torchx/docs/source` for role[0]=python.


torchx 2023-02-19 07:59:09 INFO     Waiting for the app to finish...


python/0 Hello, your name!


torchx 2023-02-19 07:59:10 INFO     Job finished: SUCCEEDED


local_docker://torchx/torchx_utils_python-rlrq42xcc0w0x


TorchX defaults to using the
[ghcr.io/pytorch/torchx](https://ghcr.io/pytorch/torchx) Docker container image
which contains the PyTorch libraries, TorchX and related dependencies.

## Distributed

TorchX's `dist.ddp` component uses
[TorchElastic](https://pytorch.org/docs/stable/distributed.elastic.html)
to manage the workers. This means you can launch multi-worker and multi-host
jobs out of the box on all of the schedulers we support.

In [6]:
%%sh
torchx run --scheduler local_docker dist.ddp --help

usage: torchx run <run args...> ddp  [--help] [--script SCRIPT] [-m M]


                                     [--image IMAGE] [--name NAME] [-h H]


                                     [--cpu CPU] [--gpu GPU] [--memMB MEMMB]


                                     [-j J] [--env ENV]


                                     [--max_retries MAX_RETRIES]


...

                        (ex.


                        type=<bind/volume>,src=/host,dst=/job[,readonly]). See


                        scheduler documentation for more info. (default: None)


  --debug DEBUG         whether to run with preset debug flags enabled


                        (default: False)


Lets create a slightly more interesting app to leverage the TorchX distributed
support.

In [7]:
%%writefile dist_app.py

import torch
import torch.distributed as dist

dist.init_process_group(backend="gloo")
print(f"I am worker {dist.get_rank()} of {dist.get_world_size()}!")

a = torch.tensor([dist.get_rank()])
dist.all_reduce(a)
print(f"all_reduce output = {a}")

Writing dist_app.py


Let launch a small job with 2 nodes and 2 worker processes per node:

In [8]:
%%sh
torchx run --scheduler local_docker dist.ddp -j 2x2 --script dist_app.py

torchx 2023-02-19 07:59:11 INFO     Checking for changes in workspace `file:///home/runner/work/torchx/torchx/docs/source`...


torchx 2023-02-19 07:59:11 INFO     To disable workspaces pass: --workspace="" from CLI or workspace=None programmatically.


torchx 2023-02-19 07:59:11 INFO     Workspace `file:///home/runner/work/torchx/torchx/docs/source` resolved to filesystem path `/home/runner/work/torchx/torchx/docs/source`


torchx 2023-02-19 07:59:12 INFO     Building workspace docker image (this may take a while)...


torchx 2023-02-19 07:59:21 INFO     Built new image `sha256:d41348164eabeb15ac718d2efb682cd2d68373c35340679d62c6e563886151b1` based on original image `ghcr.io/pytorch/torchx:0.5.0dev0` and changes in workspace `file:///home/runner/work/torchx/torchx/docs/source` for role[0]=dist_app.


...

dist_app/0 [1]:all_reduce output = tensor([6])


dist_app/0 [0]:I am worker 0 of 4!


dist_app/0 [0]:all_reduce output = tensor([6])


torchx 2023-02-19 07:59:30 INFO     Job finished: SUCCEEDED


local_docker://torchx/dist_app-j1tcp51149w0gd


## Workspaces / Patching

For each scheduler there's a concept of an `image`. For `local_cwd` and `slurm`
it uses the current working directory. For container based schedulers such as
`local_docker`, `kubernetes` and `aws_batch` it uses a docker container.

To provide the same environment between local and remote jobs, TorchX CLI uses
workspaces to automatically patch images for remote jobs on a per scheduler
basis.

When you launch a job via `torchx run` it'll overlay the current directory on
top of the provided image so your code is available in the launched job.

For `docker` based schedulers you'll need a local docker daemon to build and
push the image to your remote docker repository.

## `.torchxconfig`

Arguments to schedulers can be specified either via a command line flag to
`torchx run -s <scheduler> -c <args>` or on a per scheduler basis via a
`.torchxconfig` file.

In [9]:
%%writefile .torchxconfig

[kubernetes]
queue=torchx
image_repo=<your docker image repository>

[slurm]
partition=torchx

Writing .torchxconfig


## Remote Schedulers

TorchX supports a large number of schedulers.
Don't see yours?
[Request it!](https://github.com/pytorch/torchx/issues/new?assignees=&labels=&template=feature-request.md)

Remote schedulers operate the exact same way the local schedulers do. The same
run command for local works out of the box on remote.

```sh
$ torchx run --scheduler slurm dist.ddp -j 2x2 --script dist_app.py
$ torchx run --scheduler kubernetes dist.ddp -j 2x2 --script dist_app.py
$ torchx run --scheduler aws_batch dist.ddp -j 2x2 --script dist_app.py
$ torchx run --scheduler ray dist.ddp -j 2x2 --script dist_app.py
```

Depending on the scheduler there may be a few extra configuration parameters so
TorchX knows where to run the job and upload built images. These can either be
set via `-c` or in the `.torchxconfig` file.

All config options:

In [10]:
%%sh
torchx runopts

local_docker:


    usage:


        [copy_env=COPY_ENV],[image_repo=IMAGE_REPO]


    optional arguments:


        copy_env=COPY_ENV (typing.List[str], None)


...

            working directory in container jobs


        host_network=HOST_NETWORK (bool, False)


            True if using the host network for jobs


        shm_size=SHM_SIZE (str, 64m)


            size of shared memory (/dev/shm) for jobs


## Custom Images

### Docker-based Schedulers

If you want more than the standard PyTorch libraries you can add custom
Dockerfile or build your own docker container and use it as the base image for
your TorchX jobs.


In [11]:
%%writefile timm_app.py

import timm

print(timm.models.resnet18())

Writing timm_app.py


In [12]:
%%writefile Dockerfile.torchx

FROM pytorch/pytorch:1.10.0-cuda11.3-cudnn8-runtime

RUN pip install timm

COPY . .

Writing Dockerfile.torchx


Once we have the Dockerfile created we can launch as normal and TorchX will
automatically build the image with the newly provided Dockerfile instead of the
default one.

In [13]:
%%sh
torchx run --scheduler local_docker utils.python --script timm_app.py

torchx 2023-02-19 07:59:32 INFO     loaded configs from /home/runner/work/torchx/torchx/docs/source/.torchxconfig


torchx 2023-02-19 07:59:32 INFO     Checking for changes in workspace `file:///home/runner/work/torchx/torchx/docs/source`...


torchx 2023-02-19 07:59:32 INFO     To disable workspaces pass: --workspace="" from CLI or workspace=None programmatically.


torchx 2023-02-19 07:59:32 INFO     Workspace `file:///home/runner/work/torchx/torchx/docs/source` resolved to filesystem path `/home/runner/work/torchx/torchx/docs/source`


torchx 2023-02-19 07:59:32 INFO     Building workspace docker image (this may take a while)...


...

python/0   (global_pool): SelectAdaptivePool2d (pool_type=avg, flatten=Flatten(start_dim=1, end_dim=-1))


python/0   (fc): Linear(in_features=512, out_features=1000, bias=True)


python/0 )


torchx 2023-02-19 08:00:49 INFO     Job finished: SUCCEEDED


local_docker://torchx/torchx_utils_python-stqk7xdfvrsdld


### Slurm

The `slurm` and `local_cwd` use the current environment so you can use `pip` and
`conda` as normal.

## Next Steps

1. Checkout other features of the [torchx CLI](cli.rst)
2. Take a look at the [list of schedulers](schedulers.rst) supported by the runner
3. Browse through the collection of [builtin components](components/overview.rst)
4. See which [ML pipeline platforms](pipelines.rst) you can run components on
5. See a [training app example](examples_apps/index.rst)